In [45]:
# would recommend using conda for the packages
import sys
import os
import io
from pgn_parser import parser, pgn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, History
import chess
import chess.pgn
import tqdm
import pandas as pd
import numpy as np


In [22]:
def board_fen_to_8x8(bf):
    our_board = list()
    long_board = list()
    long_board2 = list()
    for s in bf.split('/'):
        long_board+=s
    for l in long_board: 
        if l.isnumeric():     
            i = int(l)
            while i>0:
                long_board2.append(0)
                i-=1
        else:
            long_board2.append(l)
    h =0
    for i in range(0,8):
        our_board.append([])
        for j in range(0,8):
            our_board[i].append([])
            our_board[i][j] = long_board2[h]
            h+=1
            
    return our_board


In [23]:
def letter_to_number(letter):
    letter = str(letter)
    color = 1
    if letter.isupper(): color = -1
    return  color*((1*(letter.lower() == 'p'))+(2*(letter.lower() == 'n'))\
            +(3*(letter.lower() == 'b'))+(4*(letter.lower() == 'r'))\
            +(5*(letter.lower() == 'q'))+(6*(letter.lower() == 'k')))

In [24]:
def board_to_nums(board):
    b2 = board.copy()
    for i in range(0,8):
        for j in range(0,8):
            b2[i][j] = letter_to_number(board[i][j])
    return b2

In [25]:
def board_to_12_array(board):
    ta = [[],[],[],[],[],[],[],[],[],[],[],[]]
    # if its a 0, add 0 to all 12 arrays
    # if a 1,2,3,4,5,6 or negs, add a 1 to the corresponding array and 0 to all others
    
    b = board.copy()
    for i in range(0,8):
        for m in range(0,12):
            ta[m].append([])
        for j in range(0,8):
            # blank space
            if b[i][j] == 0:
                for k in range(0,12):
                    ta[k][i].append(0)
            # piece is white
            elif b[i][j] < 0:
                ta[-1*(b[i][j])+5][i].append(1)
                for k in range(0,12):
                    if k != (-1*(b[i][j])+5):
                        ta[k][i].append(0)
            # piece is black   
            else:
                ta[b[i][j]-1][i].append(1)
                for k in range(0,12):
                    if k != (b[i][j]-1):
                        ta[k][i].append(0)
        
            
    return ta

In [26]:
# array to hold all input positions
boards = []

gamepgn=open("lichess_12_2022.pgn",'r')
#game = chess.pgn.read_game(gamepgn)
#board = game.board()

# go through first 1000 games from dataset
for i in range(0,100):
    game = chess.pgn.read_game(gamepgn)
    board = game.board()
    # after reading the next game, take each position's FEN and convert to board array
    for move in game.mainline_moves():
        b = {}
        board.push(move)
        b['board'] = board_to_nums(board_fen_to_8x8(board.board_fen()))
        b['oneHboards'] = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
        boards.append(b)
        
boards = pd.DataFrame(boards)

print(len(boards))
print(boards.loc[0]["board"][7])
print(boards.loc[0]["oneHboards"])

8236
[-4, -2, -3, -5, -6, -3, -2, -4]
[[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 

In [27]:
num_games = 100
rows = []
with open("lichess_12_2022.pgn") as pgn:
    for game in tqdm.tqdm(range(num_games)):
        game = chess.pgn.read_game(pgn)
        for move in game.mainline_moves():
            u=list(str(move.uci()).strip(""))
            squares=[]
            for i in range(0,4):
                if u[i].isnumeric():
                    squares.append(float(u[i])/8)
                else:
                    squares.append(float(ord(u[i])-96)/8) 
            row={}
            row['moves'] = squares
            rows.append(row)
moves = pd.DataFrame(rows)

master = pd.concat([boards, moves], axis=1, join="inner")
print(master)

master.to_csv("loadedgames.csv", index = False)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 684.93it/s]


                                                  board   
0     [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 1, 1, 1, ...  \
1     [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 0, 1, 1, ...   
2     [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 0, 1, 1, ...   
3     [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 1, 0, 1, 1, ...   
4     [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 1, 0, 1, 1, ...   
...                                                 ...   
8231  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 6, ...   
8232  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 6, ...   
8233  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 6, ...   
8234  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 6, ...   
8235  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 6, ...   

                                             oneHboards   
0     [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1,...  \
1     [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
2     [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
3     [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...

In [42]:
inp = keras.layers.Input(shape=(12,8,8))

m = keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv1")(inp)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="same", name="mPool1")(m)
m = keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv2")(m)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="same", name="mPool2")(m)
m = keras.layers.Flatten()(m)
m = keras.layers.Dense(64, activation='relu')(m)
output_move = keras.layers.Dense(4, activation='sigmoid')(m)

model = keras.Model(inputs=inp, outputs=output_move, name="pawn_star_1")
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.mse, metrics=None)
model.summary()

Model: "pawn_star_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 12, 8, 8)]        0         
                                                                 
 conv1 (Conv2D)              (None, 12, 8, 32)         2336      
                                                                 
 mPool1 (MaxPooling2D)       (None, 6, 4, 32)          0         
                                                                 
 conv2 (Conv2D)              (None, 6, 4, 32)          9248      
                                                                 
 mPool2 (MaxPooling2D)       (None, 3, 2, 32)          0         
                                                                 
 flatten_4 (Flatten)         (None, 192)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                

In [ ]:
train_boards = master["oneHboards"].to_list()
train_moves = master["moves"].to_list()

#earlystop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=250, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

callback = keras.callbacks.ModelCheckpoint(filepath="checkpoints/cp-{epoch:04d}.hdf5",
                                           verbose=1, 
                                           save_freq=116, 
                                           save_weights_only=True, 
                                           #save_best_only=True,
                                           monitor="val_loss", 
                                           mode="max")

inpx = tf.convert_to_tensor(train_boards)
inpy = tf.convert_to_tensor(train_moves)
model.fit(x=inpx, 
          y=inpy, 
          batch_size=64, 
          epochs=100, 
          callbacks=[callback],
          validation_split=0.1)

Epoch 1/100
112/116 [===========================>..] - ETA: 0s - loss: 0.0185
Epoch 1: saving model to checkpoints\cp-0001.hdf5
116/116 [==============================] - 1s 6ms/step - loss: 0.0185 - val_loss: 0.1129
Epoch 2/100
111/116 [===========================>..] - ETA: 0s - loss: 0.0181
Epoch 2: saving model to checkpoints\cp-0002.hdf5
116/116 [==============================] - 1s 6ms/step - loss: 0.0182 - val_loss: 0.1118
Epoch 3/100
113/116 [============================>.] - ETA: 0s - loss: 0.0181
Epoch 3: saving model to checkpoints\cp-0003.hdf5
116/116 [==============================] - 1s 6ms/step - loss: 0.0181 - val_loss: 0.1118
Epoch 4/100
111/116 [===========================>..] - ETA: 0s - loss: 0.0179
Epoch 4: saving model to checkpoints\cp-0004.hdf5
116/116 [==============================] - 1s 6ms/step - loss: 0.0180 - val_loss: 0.1101
Epoch 5/100
113/116 [============================>.] - ETA: 0s - loss: 0.0178
Epoch 5: saving model to checkpoints\cp-0005.hdf5
116/